In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from nbformat.validator import normalize
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Data Pipeline

**Preprocessing Pipeline**

In [3]:
transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(
         mean=(0.1307,),
         std=(0.3081,)
     )
     ]
)

In [4]:
#Download dataset
train_data=datasets.MNIST(root=r'C:\Users\User\Downloads\60 days of python\day-32\data',train=True,transform=transform,download=True)

In [5]:
test_data=datasets.MNIST(root=r'C:\Users\User\Downloads\60 days of python\day-32\data',train=False,transform=transform,download=True)

In [6]:
train_loader=DataLoader(
    train_data,
    batch_size=64,
    shuffle=True,
)
test_loader=DataLoader(
    test_data,
    batch_size=64,
    shuffle=False,
)

In [7]:
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.w = nn.Parameter(torch.randn(input_size))
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, x):
        x = x @ self.w + self.b
        return x

In [8]:
input_size=28*28
sample_input=torch.randn(input_size)
Perceptron_model=Perceptron(input_size=input_size)
output=Perceptron_model(sample_input)
print(output)

tensor([32.3186], grad_fn=<AddBackward0>)


In [9]:
"""Acticavation function
function=relu(z)=max(0,z)
"""
class ReLU(nn.Module):
    def __init__(self):
        super(ReLU, self).__init__()
    def forward(self, x):
        return torch.max(torch.tensor(0.0),x)

relu=ReLU()
output=relu(output)
print(output)

tensor([32.3186], grad_fn=<MaximumBackward0>)


In [10]:
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(Linear, self).__init__()

        self.perceptrons = nn.ModuleList([
            Perceptron(input_size) for _ in range(output_size)
        ])

    def forward(self, x):
        outputs = [
            perceptron(x) for perceptron in self.perceptrons
        ]
        outputs = torch.stack(outputs, dim=1)
        return outputs

In [11]:
class DigitClassifier(nn.Module):
    def __init__(self, input_size=28 * 28, output_size=10):
        super(DigitClassifier, self).__init__()
        self.fc1 = Linear(input_size, 256)
        self.fc2 = Linear(256, 128)
        self.fc3 = Linear(128, output_size)
        self.relu = ReLU()

    def forward(self, x):
        # Let say x = [1, 2, 3, ....]   shape:  [784]
        x = x.view(-1, input_size)
        # Let say x = [1, 2, 3, ....]   shape:  [1, 784]
        x = self.fc1(x)
        # Let say x = [-1, 2.2, 3.3, ...]   shape:  [1, 128]
        x = self.relu(x)
        # Let say x = [0, 2.2, 3.3, ...]   shape:  [1, 128]
        x = self.fc2(x)
        # Let say x = [-.50, 4.2, -3.3, ...]   shape:  [1, 64]
        x = self.relu(x)
        # Let say x = [0, 4.2, 0, ...]   shape:  [1, 64]
        x = self.fc3(x)
        # Let say x = [5, -4.2, .3, ...]   shape:  [1, 10]
        return x

In [12]:
model = DigitClassifier(input_size=28 * 28, output_size=10).to(device)
sample_input = sample_input.to(device)

output = model(sample_input)
print(output.shape)
print(output)

torch.Size([1, 10])
tensor([[-2055.8979, -1664.2817,   549.7171, -3464.9875, -3135.2490, -1911.2336,
          2120.1030,   294.2802,   438.4024,   369.2884]],
       grad_fn=<StackBackward0>)


In [13]:
print(output)

_, predicted = torch.max(output, 1)
print("Class label:", predicted)

tensor([[-2055.8979, -1664.2817,   549.7171, -3464.9875, -3135.2490, -1911.2336,
          2120.1030,   294.2802,   438.4024,   369.2884]],
       grad_fn=<StackBackward0>)
Class label: tensor([6])


In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [15]:
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch [1/2], Step [1/938], Loss: 3681.9475
Epoch [1/2], Step [101/938], Loss: 523.3168
Epoch [1/2], Step [201/938], Loss: 448.0367
Epoch [1/2], Step [301/938], Loss: 500.4863
Epoch [1/2], Step [401/938], Loss: 143.5962
Epoch [1/2], Step [501/938], Loss: 136.2161
Epoch [1/2], Step [601/938], Loss: 151.8893
Epoch [1/2], Step [701/938], Loss: 108.4171
Epoch [1/2], Step [801/938], Loss: 178.2355
Epoch [1/2], Step [901/938], Loss: 31.4161
Epoch 1, Loss: 261.3497155846309
Epoch [2/2], Step [1/938], Loss: 42.8234
Epoch [2/2], Step [101/938], Loss: 34.6274
Epoch [2/2], Step [201/938], Loss: 114.2720
Epoch [2/2], Step [301/938], Loss: 75.5140
Epoch [2/2], Step [401/938], Loss: 41.5759
Epoch [2/2], Step [501/938], Loss: 77.2684
Epoch [2/2], Step [601/938], Loss: 80.2570
Epoch [2/2], Step [701/938], Loss: 43.3021
Epoch [2/2], Step [801/938], Loss: 24.4755
Epoch [2/2], Step [901/938], Loss: 38.7707
Epoch 2, Loss: 65.95087028591871


In [16]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 90.62%


In [46]:
x=torch.tensor([43,52,11],dtype=torch.float)
perceptron=[Perceptron(input_size=3),Perceptron(input_size=3),Perceptron(input_size=3)]
y=[i(x) for i in perceptron]
print(y)

[tensor([53.3585], grad_fn=<AddBackward0>), tensor([-71.8283], grad_fn=<AddBackward0>), tensor([73.5812], grad_fn=<AddBackward0>)]


In [47]:
output=torch.stack(y).shape
print(output)


torch.Size([3, 1])


In [56]:
x=torch.tensor([[1],[4],[5]])
print(torch.stack(list(x),dim=1).shape)

torch.Size([1, 3])
